In [ ]:
import os
import sys
sys.path = [p for p in sys.path if p not in ("", ".", "/Users/yilu/repos/reflexio")]
sys.path.insert(0, '/Users/yilu/repos/reflexio/reflexio/reflexio_client')

## Login and setup

In [ ]:
import reflexio

In [ ]:
reflexio.__file__

In [ ]:
from reflexio import (
    InteractionData,
    PublishUserInteractionRequest,
    SearchInteractionRequest,
    SearchUserProfileRequest,
    DeleteUserProfileRequest,
    DeleteUserInteractionRequest,
    UserActionType)

client = reflexio.ReflexioClient(url_endpoint="http://localhost:8081/")
token = client.login("local_supabase", "s")

In [ ]:
remote_client = reflexio.ReflexioClient(url_endpoint="https://www.reflexio.ai/")
_ = remote_client.login("local_supabase", "s")

In [ ]:
remote_config = remote_client.get_config()

In [ ]:
remote_config

In [ ]:
client.set_config(remote_config)

In [ ]:
client.api_key

In [ ]:
client = reflexio.ReflexioClient(url_endpoint="http://ext-agenticmem-development-1613773270.us-west-2.elb.amazonaws.com")
token = client.login("code_interviewer", "s")
# client.api_key = token.api_key

from reflexio import (
    InteractionData,
    PublishUserInteractionRequest,
    SearchInteractionRequest,
    SearchUserProfileRequest,
    DeleteUserProfileRequest,
    DeleteUserInteractionRequest,
    UserActionType)

In [ ]:
from reflexio.client import ReflexioClient
from reflexio_commons.api_schema.service_schemas import InteractionData, UserActionType

url_endpoint = "http://agenticmem-alb-814973042.us-east-1.elb.amazonaws.com/"

client = reflexio.ReflexioClient(url_endpoint=url_endpoint)
client.login("code_interviewer", "s")

client.publish_interaction(
    user_id="your_user_id_test",
    interactions=[
        InteractionData(
            role="User",  # agent, student or other that you can define to match your prompt
            content="Can I make an order",
        ),
        InteractionData(
            role="Assistent",  # agent, student or other that you can define to match your prompt
            content="""Sure, what would you like to have?""",
            shadow_content="""Sure, would you like to another BBQ chikchen pizza that you ordered last time or try something else?""",
            user_action=UserActionType.NONE,  # Optional, defaults to NONE
            user_action_description="",  # Optional, defaults to empty string
            interacted_image_url="",  # Optional, defaults to empty string
        ),
    ],
    source="test",  # optional, mark interaction if needed. can be used for filtering when search user profiles
    agent_version="1.0",
    request_group="group_test",
)
 

# Config management
Use admin portal if hosted locally, otherwise use API to get and set configs for Reflexio

In [ ]:
config = client.get_config()
config.model_dump()

In [ ]:
config = client.get_config()
client.set_config(config)

In [ ]:
local_client.get_config()

### Set config

In [ ]:
from reflexio import ProfileExtractorConfig, AgentFeedbackConfig, FeedbackAggregatorConfig, AgentSuccessConfig, ToolUseConfig
agent_context_prompt = "this is an interview agent"
profile_extractor_configs=[
    ProfileExtractorConfig(
        extractor_name="test_extractor",
        context_prompt="""
Conversation between sales agent and user, extract any information from the interaction if contains any information listed under definition
""",
        profile_content_definition_prompt="""
name, age, intent of the conversations
""",
        metadata_definition_prompt="""
choice of ['basic_info', 'conversation_intent']
""",
    )
]

agent_feedback_configs=[
    AgentFeedbackConfig(
        feedback_name="test_config_feedback",
        feedback_definition_prompt="""
feedback should be something user told you to do differently in the next session. something sales rep did that makes user not satisfied.
feedback content is what agent should do differently in the next session based on the conversation history and be actionable as much as possible.
for example:
if user mentiones "I don't like the way you talked to me", summarize conversation history and feedback content should be what is the way agent talk which is not preferred by user.
""",
        feedback_aggregator_config=FeedbackAggregatorConfig(
            min_feedback_threshold=3,
        ),
    )
]

agent_success_configs = [
    AgentSuccessConfig(
        evaluation_name="test_agent_success",
        success_definition_prompt="does the teacher answer student question and student is able to move forward",
        tool_can_use=[
            ToolUseConfig(
                tool_name="search",
                tool_description="Search for information",
            )
        ],
        action_space=["search", "respond"],
    )
]

config.agent_context_prompt = agent_context_prompt
config.profile_extractor_configs = profile_extractor_configs
config.agent_feedback_configs = agent_feedback_configs
config.agent_success_configs = agent_success_configs


client.set_config(config)

In [ ]:
config = client.get_config()
cur_config = config.model_dump()

In [ ]:
client.set_config(config)

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

conversations = pd.read_json('/Users/yilu/repos/reflexio/reflexio/tests/evaluation/evaluation_data/customer_support.jsonl', lines=True)
conversations

# Publish user interactions with the system
User interactions are all the activities that user has with either the agent or the system in general. it can contain text user says to the agent, button clicks or contextual information when user takes specific action, such as what the user is looking at when click the button

## Publish text profiles

In [ ]:
client.publish_interaction(
    user_id="your_user_id_test",
    interactions=[
        InteractionData(
            role="User", # agent, student or other that you can define to match your prompt
            content="Can I make an order",
        ),
        InteractionData(
            role="Assistent", # agent, student or other that you can define to match your prompt
            content="""Sure, what would you like to have?""",
            shadow_content="""Sure, would you like to another BBQ chikchen pizza that you ordered last time or try something else?""",
            user_action=UserActionType.NONE,  # Optional, defaults to NONE
            user_action_description="",  # Optional, defaults to empty string
            interacted_image_url=""  # Optional, defaults to empty string
        )
    ],
    source = "test", # optional, mark interaction if needed. can be used for filtering when search user profiles
    agent_version = "1.0",
    request_group="group_3"
)

In [ ]:
interactions = []
for _, row in conversations[:10].iterrows():
    interactions.append(
        InteractionData(
            role=row['role'],
            content = row['content']
        )
    )

client.publish_interaction(
    user_id="your_user_id_1",
    interactions=interactions,
    source = "source1",
    agent_version = "1.0",
    request_group="conversation_id_1"
)

In [ ]:
interactions = []
for _, row in conversations[8:16].iterrows():
    interactions.append(
        InteractionData(
            role=row['role'],
            content = row['content']
        )
    )

client.publish_interaction(
    user_id="your_user_id_2",
    interactions=interactions,
    source = "source1",
    agent_version = "1.0",
    request_group="conversation_id_2"
)

In [ ]:
interactions = []
for _, row in conversations[16:].iterrows():
    interactions.append(
        InteractionData(
            role=row['role'],
            content = row['content']
        )
    )

client.publish_interaction(
    user_id="your_user_id_1",
    interactions=interactions,
    source = "source1",
    agent_version = "1.0",
    request_group="conversation_id_1"
)

In [ ]:
client.manual_profile_generation(
    user_id="your_user_id_7",
    extractor_names=['implicit_extractor']
)   

In [ ]:
client.manual_feedback_generation(
    agent_version="1.0"
)

In [ ]:
client.add_feedbacks(
    feedbacks=[
        {
            "agent_version": "1.0",
            "feedback_name": "manual_addition",
            "feedback_content": "Agent should provide more concise responses",
        }
    ]
)

## Publish visual interaction

In [ ]:
client.publish_interaction(
    request_id="your_request_id", # for attribution for profile sources
    user_id="your_user_id",
    interaction_requests = [
        InteractionRequest(
            timestamp=datetime.now(),
            content="I like this sushi",
            interacted_image_url="image url user is looking at when say this"
        )
    ])

In [ ]:
import base64

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


# Path to your image
image_path = "./data/sushi.png"

# Getting the base64 string
base64_image = encode_image(image_path)

# Publish image
client.publish_interaction(
    request_id="your_request_id", # for attribution for profile sources
    user_id="your_user_id",
    interaction_requests = [
        InteractionRequest(
            content="I like this",
            image_encoding=base64_image,
        )
    ])

## Publish other non-chatbot interactions

In [ ]:
client.publish_interaction(
    request_id="your_request_id", # for attribution for profile sources
    user_id="your_user_id",
    interaction_requests = [
        InteractionRequest(
            timestamp=datetime.now(),
            user_action=UserActionType.CLICK,
            user_action_description="book restaurant button"
        )
    ])

# Search interactions and profiles
Search for relevant interaction current user activity to enrich the context of LLM agent for better personalization and long term memory

# Interactions

List all interactions

In [ ]:
interactions = client.get_interactions(
    request = {
        "user_id": "69d12754-4d38-4078-b4ef-9cbd906e3b82::72771854-ebb7-4cf7-bf86-0ecb3d1239a8",
        "top_k": 4
    }
)
interactions.model_dump()

Search interactions

In [ ]:
response = client.search_interactions(
    user_id="69d12754-4d38-4078-b4ef-9cbd906e3b82::72771854-ebb7-4cf7-bf86-0ecb3d1239a8",
    query="food preference",
    top_k=5,
    most_recent_k=10
)
response

# Profiles

Get profile change logs (create, update, delete, mention operation)

In [ ]:
logs = client.get_profile_change_log()
logs.model_dump()

In [ ]:
profiles = client.get_profiles(
    request = {
        "user_id": "69d12754-4d38-4078-b4ef-9cbd906e3b82::72771854-ebb7-4cf7-bf86-0ecb3d1239a8",
        "top_k": 4
    }
)
profiles.model_dump()

Search profiles

In [ ]:
response = client.search_profiles(
    user_id="your_user_id_3",
    query="name",
    threshold=0.1,
    top_k=5
)
response

In [ ]:
response = client.search_profiles(
    user_id="your_user_id_3",
    query="name",
    threshold=0.1,
    top_k=5,
    extractor_name="fact_extractor"
)
response

**Manually trigger profile generation**

In [ ]:
client.rerun_profile_generation(
    user_id="your_user_id1"
)

In [ ]:
# trigger a specific extractor
client.rerun_profile_generation(
    user_id="your_user_id1",
    extractor_names=['extractor_1']
)

In [ ]:
client.manual_profile_generation(
    user_id="your_user_id_test"
)

# Agentic feedback

## Get all raw feedbacks

**List all raw user feedbacks**

In [ ]:
raw_feedbacks = client.get_raw_feedbacks()
raw_feedbacks

**Get all aggregated feedback**

In [ ]:
feedbacks = client.get_feedbacks()
feedbacks

**Trigger feedback aggregation manually**

In [ ]:
client.run_feedback_aggregation(agent_version='1.0', feedback_name="Test Config Feedback")

**Get feedback generated and filter by status**

In [ ]:
from reflexio import Status, FeedbackStatus

client.get_feedbacks(feedback_name="test_config_feedback", feedback_status_filter=FeedbackStatus.PENDING)

**Manually add raw feedbacks**

In [ ]:
from reflexio import RawFeedback

client.add_raw_feedback([RawFeedback(agent_version="1.0", request_id="manual", feedback_name='test_config_feedback', feedback_content="pretty simple answer"),
                        RawFeedback(agent_version="1.0", request_id="manual", feedback_name='test_config_feedback', feedback_content="start with easy response"),
                        RawFeedback(agent_version="1.0", request_id="manual", feedback_name='test_config_feedback', feedback_content="response should be concise"),
                        RawFeedback(agent_version="1.0", request_id="manual", feedback_name='test_config_feedback', feedback_content="prefer shorter response")])

In [ ]:
client.manual_feedback_generation(
    agent_version="1.0"
)

# Delete user interaction or profile

In [ ]:
# delete interaction
# you can locate the interaction id from the web UI
client.delete_interaction(
    user_id="your_user_id",
    interaction_id="your_user_id_your_request_id_04eb5352"
)

In [ ]:
# delete user profile
# you can locate the user profile id from the web UI
lient.delete_profile(
    user_id="your_user_id",
    profile_id='5ce7f859-0517-4fe4-9c6a-5145a4a73f04'
)